# Calculate normalised enrichment scores of ConsensusTME gene sets

Script related to figure 3b

### Aim:
- Calculate TCGA OV NES of ConsensusTME Gene Sets

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load expression data

In [ ]:
mix <- "../../Data/1/TCGAov_RNAseq.txt"

In [3]:
data <- read.csv(mix,
                 sep='\t',
                 header=T,
                 row.names='GeneSymbol')

In [4]:
head(data)

,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,TCGA.09.0367.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
A1BG,66.4695,65.5664,41.6412,187.0368,23.9295,32.8123,46.3418,162.8522,88.5518,192.7354,⋯,63.7012,99.1485,38.3684,140.3668,94.7103,70.4394,43.1736,33.6854,27.9604,16.2092
A1BG-AS1,36.3243,28.1315,23.2465,114.6008,10.4957,20.1900,17.2007,83.0906,65.6758,55.1265,⋯,30.3602,71.1797,26.0239,77.2415,61.4890,106.1623,47.8864,25.3482,17.9853,42.9450
A1CF,0.0000,0.0000,0.3310,0.0000,0.0000,0.0000,0.0000,0.9034,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,1.9057,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A2M,5899.8279,9384.4401,3350.4207,1455.2316,3999.3792,3224.5797,1986.1593,1487.3247,10557.8325,4603.9013,⋯,10264.1373,7220.7664,7200.1144,1819.9428,6179.7358,5365.7972,8773.5936,10006.7877,1888.1746,4205.6055
A2M-AS1,118.4566,111.0026,71.3613,67.8607,52.1501,224.7582,182.0650,120.2838,53.6419,92.9112,⋯,146.6130,45.8648,64.4638,28.5565,100.6605,23.5625,84.2153,582.3492,59.8084,143.4910
A2ML1,7.5289,54.6875,5.6263,5.0883,3.3148,3.7421,6.2305,278.2486,4.7763,653.6058,⋯,1.7993,4.6124,0.4768,17.1510,15.6109,2.4461,98.3162,269.5768,4.5092,224.5290


In [5]:
data_mtx <- as.matrix(data)

## Load ConsensusTME gene sets

In [6]:
ConTME_GeneSets <- read.csv('../../Data/3/ConsensusTME_GeneSets.txt',
                            sep='\t',
                            header=T)

In [7]:
head(ConTME_GeneSets)

B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
BLNK,CTSW,BCL2A1,CD93,ABHD2,ABCA6,ACTR2,ACTR2,AIF1,ATP6V1C1,AIF1,APOBEC3G,BCL2A1,CD27,AIM2,BCL11B,APOBEC3G,CCR7
CD1C,GZMA,BIRC3,CDH5,BCL2A1,ASPN,ATG7,CCL19,ATP6V1C1,CCL4,AKAP13,CCL4,C5AR1,CD38,ARHGAP15,BIN2,CCL4,CD2
CD37,GZMB,CCL19,EMCN,CCL4,COL3A1,ATP6V1C1,CCL4,CLEC10A,CD84,C5AR1,CCR5,CSF2RB,CYBA,BATF,CCL4,CCR2,CD247
CD53,GZMH,CCL4,ERG,CCR1,DCN,BCL2A1,CCR1,CLEC4A,LCP2,CCR2,CD2,CSF3R,ENTPD1,BCL11B,CCR5,CCR5,CD27
CD69,KLRB1,CCR2,KDR,CD69,GREM1,C5AR1,CCR7,FGR,MMP9,CD244,CD244,FCGR3B,HSPA6,CCL4,CCR7,CD2,CD37
CD80,KLRK1,CCR5,PCDH12,CSF2RB,ISLR,CCL19,CD38,HCK,SAMSN1,CD37,CD247,FPR1,MTDH,CCR2,CD2,CD244,CD3E


In [8]:
ConTME_GeneSets <- convertRowsToList(t(ConTME_GeneSets))

## Run ssGSEA

In [9]:
con_NES <- gsva(data_mtx,
                ConTME_GeneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 18 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [10]:
con_NES <- data.frame("Term"=rownames(con_NES),
                      con_NES)

rownames(con_NES) <- NULL

In [11]:
head(con_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
B_cells,0.11609382,0.22290548,-0.05337479,-0.2420562,0.04399216,-0.22959159,-0.14949609,-0.195321321,0.1985960,⋯,-0.03760044,0.09318269,0.013940235,-0.09427961,0.10628745,0.06997789,-0.14213896,-0.1848621,-0.06796385,-0.16457015
Cytotoxic_cells,0.01519822,0.14255561,-0.31383134,-0.4792572,-0.05918070,-0.50135244,-0.37791805,-0.531799891,-0.2320777,⋯,-0.41457034,-0.20378344,-0.232954372,-0.34666043,-0.25780100,-0.24272572,-0.52377136,-0.4904037,-0.31050044,-0.53384083
Dendritic_cells,0.17787153,0.26478408,-0.01250251,-0.2414475,0.12695363,-0.21268284,-0.13440659,-0.231657069,0.2262739,⋯,0.02038188,0.14266558,0.015431621,-0.06713982,0.03522807,0.12372735,-0.17542054,-0.1550452,-0.10624514,-0.18362574
Endothelial,-0.02197883,0.01835045,-0.10314986,-0.1330033,-0.03542567,-0.03784834,-0.17278292,-0.118947740,-0.0334327,⋯,0.21094340,-0.00492446,-0.098267914,-0.16081016,-0.01614860,0.10565120,-0.13273756,-0.0738101,-0.17564988,-0.12621419
Eosinophils,0.16327315,0.25433465,-0.01236931,-0.1773189,0.13760869,-0.14256235,-0.10732840,-0.146863847,0.2055240,⋯,0.01980439,0.10607730,0.048563111,0.01011209,0.02630588,0.15959726,-0.06902823,-0.1627376,-0.03584268,-0.14502728
Fibroblasts,-0.02636739,0.12083934,-0.08078278,-0.1019093,-0.02378641,0.04346422,0.01001716,0.002227394,-0.1421594,⋯,0.25215143,0.04334572,-0.003044061,0.07866700,0.12107945,0.23471212,-0.01592671,-0.1975909,-0.02591576,-0.03654063


### Save NES

In [12]:
write.table(con_NES,
            file='../../Data/3/ConsensusTME_TCGAov_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script